In [226]:
import uol_redacoes_xml
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import mac_morpho
import nltk.corpus
import pandas as pd
import regex as re

In [193]:
import nltk.corpus

In [194]:
essays = uol_redacoes_xml.load()

Separar as redações por frase para analise semantica

algoritmo que procura a substring ',' nos erros, se ele achar na primeira posição da tupla de erros e não achar na segunda posição, quer dizer que a virgula foi usada erroneamente. Caso contrário, quer dizer que ou a correção acrescentou mais palavras e isso precisou adcionar virgular, ou foram adcionados uma virgula

Gerando um dataFrame que vincula o erro a possíveis frases. Possíveis frases pois a correção uol não retorna o local do erro.Portanto, o erro foi buscado pela palavra contida em error[0].

In [112]:
dfs = []
df = pd.DataFrame(columns=['texto','frase', 'erro', 'correção'])
texto = 0
for essay in essays: 
    sentences = sent_tokenize(essay.text)
    for error in essay.errors:  
        indice_first = error[0].find(',')
        indice_second = error[1].find(',')
        if indice_first != -1 and indice_second != -1:
            pass
        elif (indice_first != -1 or indice_second != -1) and ( abs( len(error[0]) - len(error[1])) < 2): 
            for setence in sentences:
                setence_error = setence.find(error[0])
                if setence_error != -1: 
                        row = {
                        'texto': texto,
                        'frase': setence,
                        'erro': error[0],
                        'correção': error[1]
                        }
                        dfs.append(pd.DataFrame(row, index=[0]))
    df = pd.concat(dfs, ignore_index=True)
    texto = texto+1

In [153]:
df

,texto,frase,erro,correção
0,0,"A liberdade de expressão, assim como o direito...",intimidade,"intimidade,"
1,0,"Porém, há casos em que essas garantias são pos...",intimidade,"intimidade,"
2,0,Em épocas de regimes autoritários como as dita...,autoritários,"autoritários,"
3,0,"Nessas épocas, as pessoas também sofriam grave...",privada,"privada,"
4,0,"Assim, o alvo dessas matérias, pode requerer s...",privada,"privada,"
...,...,...,...,...
2307,2021,Apesar de a favela conter grande parte da viol...,",",
2308,2021,Devemos ter consciência de que muitos que mora...,",",
2309,2021,"Pelo contrário, enfrentaram muitos obstáculos ...",",",
2310,2021,"Enfim, para um país encontrar progresso, na po...",",",


In [82]:
# Frase tokenizada
frase = df['frase'][0]

# Tokenizar a frase em palavras
palavras_tokenizadas = word_tokenize(frase, language='portuguese')

# Obter a lista de palavras etiquetadas
palavras_etiquetadas = mac_morpho.tagged_words()

# Iterar sobre as palavras tokenizadas e obter suas tags
for palavra in palavras_tokenizadas:
    # Procurar a palavra na lista de palavras etiquetadas
    for palavra_etiquetada in palavras_etiquetadas:
        if palavra == palavra_etiquetada[0]:
            # Imprimir a palavra e sua tag correspondente
            print(f'{palavra}: {palavra_etiquetada[1]}')
            break

A: ART
liberdade: N
de: PREP
expressão: N
,: ,
assim: PDEN
como: KS
o: ART
direito: N
à: KS
intimidade: N
são: V
conquistas: N
históricas: ADJ
.: .


Separar elementos com a mesma função sintática

Antes de conjunções adversativas **x**

Separar orações intercaladas

aposto **x**

Separar expressões explicativas

Isolar o vocativo

Isolar datas

Marcar zeugma

Separar as orações subordinadas adverbiais

Separar as orações subordinadas adverbiais deslocadas

Não separar o sujeito do predicado **x**

Não separar o verbo do complemento **x**

In [245]:
def count_virgula(palavra1,palavra2,frase):
    if palavra1 == None or palavra2  == None or palavra1 == '' or  palavra2 == '':
        return 0
    posicao_gato = frase.find(palavra1)
    posicao_pulou = frase.find(palavra2)

    if posicao_gato != -1 and posicao_pulou != -1:
        substring = frase[posicao_gato:posicao_pulou]
        numero_virgulas = substring.count(',')
        return numero_virgulas
    else:
        return -1

In [210]:
def sujeito_pedicado(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    # Identificar sujeito e predicado
    sujeito = ""
    predicado = ""

    for token in doc:
        if "subj" in token.dep_ and sujeito == "":
            sujeito += token.text
        elif "ROOT" in token.dep_ and predicado == "":
            predicado += token.text

    return {'sujeito': sujeito, 'predicado': predicado}

In [274]:
def error_virgula_suj_pred(frase):
    senter = sujeito_pedicado(frase)

    num = count_virgula(senter['sujeito'],senter['predicado'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [212]:
def identificar_verbo_complemento(frase):
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(frase)
    
    verb = None
    obj = None
    
    for token in doc:
        if "VERB" in token.pos_:
            verb = token.text
            for child in token.children:
                if "obj" in child.dep_:
                    obj = child.text
                    break
    
    return {'verbo': verb, 'obj': obj}


In [213]:
def error_virgula_verbo(frase):
    senter = identificar_verbo_complemento(frase)
    num = count_virgula(senter['verbo'],senter['obj'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [270]:
def error_virgula_Conj_adj(frase):
    conjuncoes_adversativas = ['mas', 'porém', 'contudo', 'todavia', 'entretanto', 'no entanto']
    frase = frase.lower() 
    palavras = frase.split()

    for i in range(1, len(palavras)):
        if palavras[i] in conjuncoes_adversativas:
            if palavras[i - 1][-1] == ',':
                return 0
            else:
                return 1

    return -1

In [304]:
def error_virgulas_aposto(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    for token in doc:
        if token.dep_ == 'appos':
            
            rela_appos = [str(child) for child in token.children]
            print(token, rela_appos)
            if rela_appos.count(',') == 2:
                return 0
            else: return 1
    else:
        return -1

In [298]:
def corretor(frase):
    error = 0
    message = ""
    
    e = error_virgula_verbo(frase)
    if e ==1:
        error += e
        message += "Comma separating verb and complement; "
    
    e = error_virgula_suj_pred(frase)
    if e == 1:
        error += e
        message += "Comma separating Subject from predicate; "
    
    e = error_virgula_Conj_adj(frase)
    if e ==1:
        error += e
        message += "Missing comma before adverbial conjunction; "
    
    e = error_virgulas_aposto(frase)
    if e ==1:
        error += e
        message += "Missing comma between the 'aposto'; "

    return error, message

In [227]:
def split_sentences(text):
    regex =  r'(?<=[.!?])\s*(?=(?:[^"]*"[^"]*")*[^"]*$)'  # Expressão regular para separar as sentenças

    sentences = re.split(regex, text)

    return sentences

In [272]:
def get_essays_corrections_tool(essays):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            if sent != "":
                error, message = corretor(sent)
                data.append({
                        'essay': i,
                        'error': error,
                        'message': message,
                        'frase': sent})
    return pd.DataFrame(data)

In [299]:
erros_df = get_essays_corrections_tool(essays[0:5])

In [307]:
erros_df

,essay,error,message,frase
0,0,0,,"A liberdade de expressão, assim como o direito..."
1,0,0,,"Quando um entra em contato com o outro, é prec..."
2,0,0,,Em épocas de regimes autoritários como as dita...
3,0,0,,"Nessas épocas, as pessoas também sofriam grave..."
4,0,0,,"Porém, há casos em que essas garantias são pos..."
5,0,1,Comma separating Subject from predicate;,"Assim, o alvo dessas matérias, pode requerer s..."
6,0,0,,"Portanto, é preciso avaliar até onde o direito..."
7,0,0,,"Dessa forma, o bem estar do ser humano deve es..."
8,1,1,Missing comma between the 'aposto';,Os direitos fundamentais contidos na Constitui...
9,1,1,Missing comma between the 'aposto';,Esse é o caso da decisão judiciária aplicada a...
